In [ ]:
import pandas as pd 
from sqlalchemy import create_engine
import numpy as np
%pylab inline

In [ ]:
#Load the 120 years history file csv and print table
athlete_events_to_read = "Resources/athlete_events.csv"
athlete_events_data_df = pd.read_csv(athlete_events_to_read)
athlete_events_data_df.head()

In [ ]:
# Check the toal number of Nan values in the datset 
print(athlete_events_data_df.isnull().sum())

In [ ]:
#Change the null values in the medals col to DWN this stands for DID NOT WIN
change_DWN = athlete_events_data_df['Medal'].fillna('DNW', inplace=True)
print(athlete_events_data_df.isnull().sum())

In [ ]:
#Check the values for NOC vs Team drop any duplicates from the list. 
print(athlete_events_data_df.loc[:, ['NOC', 'Team']].drop_duplicates()['NOC'].value_counts().head())

In [ ]:
# Read in the csv for NOC countrys. 
noc_to_read = "./Resources/noc_regions.csv"
noc_data_df = pd.read_csv(noc_to_read)
noc_data_df.head()

In [ ]:
# Drop the notes col. 
noc_data_df.drop('notes', axis = 1 , inplace = True)
noc_data_df
noc_data_df.head()

In [ ]:
# Merge both data sets togethere. 
athlete_events_merage = athlete_events_data_df.merge(noc_data_df,
                                left_on = 'NOC',
                                right_on = 'NOC',
                                how = 'left')
# Find and drop any Null values and duplicates.
athlete_events_merage.loc[athlete_events_merage['region'].isnull(),['NOC', 'Team']].drop_duplicates()

In [ ]:
# 
athlete_events_merage['region'] = np.where(athlete_events_merage['NOC']=='SGP', 'Singapore', athlete_events_merage['region'])
athlete_events_merage['region'] = np.where(athlete_events_merage['NOC']=='ROT', 'Refugee Olympic Athletes', athlete_events_merage['region'])
athlete_events_merage['region'] = np.where(athlete_events_merage['NOC']=='UNK', 'Unknown', athlete_events_merage['region'])
athlete_events_merage['region'] = np.where(athlete_events_merage['NOC']=='TUV', 'Tuvalu', athlete_events_merage['region'])

In [ ]:
athlete_events_merage.drop('Team', axis = 1, inplace = True)
athlete_events_merage.rename(columns = {'region': 'Team'}, inplace = True)

In [ ]:
athlete_events_merage.head(5)

In [ ]:
w_gdp_to_read = "Resources\w_gdp.xls"
w_gdp_df = pd.read_excel(w_gdp_to_read)
w_gdp_df.head()

In [ ]:
drop_rows = w_gdp_df.drop([0,1])
set_new_head = drop_rows.rename(columns=drop_rows.iloc[0]).drop(drop_rows.index[0])
reset_index = set_new_head.reset_index()
w_gdp_df = reset_index.drop(columns=['index'])


In [ ]:
w_gdp_df.drop(['Indicator Name', 'Indicator Code'], axis = 1, inplace = True)
w_gdp_df = pd.melt(w_gdp_df, id_vars = ['Country Name', 'Country Code'], var_name = 'Year', value_name = 'GDP')
w_gdp_df['Year'] = pd.to_numeric(w_gdp_df['Year'])
w_gdp_df.head()

In [ ]:
len(list(set(athlete_events_merage['NOC'].unique()) - set(w_gdp_df['Country Code'].unique())))

In [ ]:
len(list(set(athlete_events_merage['Team'].unique()) - set(w_gdp_df['Country Name'].unique())))

In [ ]:
athlete_events_ccode = athlete_events_merage.merge(w_gdp_df[['Country Name', 'Country Code']].drop_duplicates(),
                                            left_on = 'Team',
                                            right_on = 'Country Name',
                                            how = 'left')

athlete_events_ccode.drop('Country Name', axis = 1, inplace = True)
athlete_merge_gdp = athlete_events_ccode.merge(w_gdp_df,
                                                left_on = ['Country Code', 'Year'],
                                                right_on = ['Country Code', 'Year'],
                                                how = 'left')

athlete_merge_gdp.drop('Country Name', axis = 1, inplace = True)
athlete_merge_gdp.head(5)

In [ ]:
world_pop_to_read = "Resources\WorldPopulation.csv"
world_pop_df = pd.read_csv(world_pop_to_read)
world_pop_df.head()

In [ ]:
world_pop_df.drop(['Indicator Name', 'Indicator Code','Unnamed: 61'], axis = 1, inplace = True)

world_pop_df = pd.melt(world_pop_df, id_vars = ['Country', 'Country Code'], var_name = 'Year', value_name = 'Population')

# Change the Year to integer type
world_pop_df['Year'] = pd.to_numeric(world_pop_df['Year'])

world_pop_df.head()

In [ ]:
complete_df = athlete_merge_gdp.merge(world_pop_df,
                                            left_on = ['Country Code', 'Year'],
                                            right_on= ['Country Code', 'Year'],
                                            how = 'left')

complete_df.drop('Country', axis = 1, inplace = True)

complete_df.head()

In [ ]:
complete_df.isnull().sum()